In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from lesview import *

In [ ]:
ctname = 'CT'
Qt0s = dict(
    CT = 1.221e-5,
    CT05 = 1.221e-6,
    CT5h = 1.221e-4,
)
depths = dict(
    CT = 50,
    CT05 = 40,
    CT5h = 80,
)
w2maxs = dict(
    CT = 0.6,
    CT05 = 0.6,
    CT5h = 0.8,
)
casenames = dict(
    c1 = 'R11_MSM97-{:s}_f0_fixdt3'.format(ctname),
    c2 = 'R11_MSM97-{:s}_Stokes6_f0_fixdt3'.format(ctname),
    c3 = 'R11_MSM97-{:s}_Stokes_f0_fixdt3'.format(ctname),
)
amp = dict(
    c1 = 0.0,
    c2 = 0.45,
    c3 = 0.8,
)
linestyles = [':','-.','--','-']
colors = ['tab:blue', 'tab:green', 'tab:orange']
labels = ['$a$ = {:4.2f} m'.format(amp[cn]) for cn in amp.keys()]
iend = 57600
filename = 'his.mp.vis.000001.{:06d}.nc'.format(iend+1)
figpath = 'R11_MSM97_{:s}_Stokes_f0'.format(ctname)
os.makedirs(figpath, exist_ok=True)

In [ ]:
labels

In [ ]:
ds = {}
for cn in casenames.keys():
    print(cn)
    datapath = os.path.join(os.path.pardir, 'ncarles', casenames[cn], filename)
    ds[cn] = NCARLESDataProfile(filepath=datapath).dataset        

In [ ]:
ds['c1']

In [ ]:
g = 9.81
rho = 1026
cp = 3991
alpha = 2e-4
NNT0 = 0.01
NN0 = alpha*g*NNT0
Qt0 = Qt0s[ctname]
h0 = 33
wstar = (alpha*g*Qt0*h0)**(1/3)
depth = depths[ctname]
def get_vars(ds, var):
    if var == 'uL':
        return (ds.data_vars['uxym']+ds.data_vars['stokes'])/wstar
    elif var == 'uE':
        return ds.data_vars['uxym']/wstar
    elif var == 'uS':
        return ds.data_vars['stokes']/wstar
    elif var == 'NN':
        return ds.data_vars['txym'].differentiate(coord='z')/NNT0
    elif var == 'wNN':
        return -ds.data_vars['uxym'].differentiate(coord='z')*ds.data_vars['stokes'].differentiate(coord='z')/NN0
    elif var == 'tNN':
        return ds.data_vars['txym'].differentiate(coord='z')/NNT0-ds.data_vars['uxym'].differentiate(coord='z')*ds.data_vars['stokes'].differentiate(coord='z')/NN0
    elif var == 'wt':
        # return (ds.data_vars['wtle']+ds.data_vars['wtsb'])/Qt0
        return ds.data_vars['wtle']/Qt0
    elif var == 'wtsb':
        return ds.data_vars['wtsb']/Qt0
    elif var == 'ww':
        return ds.data_vars['wps']/wstar**2
    elif var == 'uu':
        return ds.data_vars['ups']/wstar**2
    elif var == 'vv':
        return ds.data_vars['vps']/wstar**2
    elif var == 'wu':
        return ds.data_vars['uwle']/wstar**2
    elif var == 'wusb':
        return ds.data_vars['uwsb']/wstar**2
    elif var == 'wv':
        return ds.data_vars['vwle']/wstar**2
    elif var == 'sk':
        return ds.data_vars['wcube']/ds.data_vars['wps']**1.5
    elif var == 'tke':
        return 0.5*(ds.data_vars['ups']+ds.data_vars['vps']+ds.data_vars['wps'].interp(zi=ds.z))/wstar**2
    else:
        raise ValueError('Variable {:s} not found.'.format(var))

In [ ]:
# tslice = slice('2000-01-01T16:00:00', '2000-01-02T00:00:00')
tslice = slice('2000-01-01T12:00:00', '2000-01-02T00:00:00')

In [ ]:
fig, axarr = plt.subplots(2, 3, sharey='row')
fig.set_size_inches(8,6)
variables = ['uL', 'NN', 'wt', 'ww', 'uu', 'sk']
xlabels = dict(
    uL = '$\overline{u}^L/w_*$, $\overline{u}/w_*$, $u^S/w_*$',
    NN = '$N^2/N^2_0$, $N_*^2/N^2_0$',
    wt = '$\overline{w^\prime b^\prime}/B_0$, $q^{sgs}_b/B_0$',
    ww = '$\overline{{w^\prime}^2}/w_*^2$, TKE$/w_*^2$',
    sk = '$\overline{{w^\prime}^3}/(\overline{{w^\prime}^2})^{3/2}$',
    uu = '$\overline{{u^\prime}^2}/w_*^2$, $\overline{{v^\prime}^2}/w_*^2$',
)
xlims = dict(
    uL = [-2,3],
    NN = [-1.5,2.5],
    wt = [-0.3,1.3],
    ww = [0,w2maxs[ctname]],
    sk = [-2.5,0.5],
    uu = [0,w2maxs[ctname]],
)
lr = 'aaaaaa'
tb = 'bbbaaa'
leftright = {'a': 'left', 'b': 'right'}
topbottom = {'a': 'top', 'b': 'bottom'}
abc = 'abcdef'
xabc = {'a': 0.05, 'b': 0.95}
yabc = {'a': 0.95, 'b': 0.05}
for k, var in enumerate(variables):
    m = k//3
    n = k%3
    ax = axarr[m,n]
    ax.axvline(x=0, color='k', linewidth=0.75)
    if var == 'uL':
        for i, cn in enumerate(casenames.keys()):
            da = get_vars(ds[cn], var)
            damean = da.sel(time=tslice).mean(dim='time').where(da.z>=-depth).mean(dim='z')
            ax.plot(damean,0,'x',color=colors[i],clip_on=False,zorder=3)
            da = get_vars(ds[cn], 'uS')
            da.sel(time=tslice).mean(dim='time').plot(ax=ax,y='z',color=colors[i],linestyle='--')
            da = get_vars(ds[cn], 'uE')
            da.sel(time=tslice).mean(dim='time').plot(ax=ax,y='z',color=colors[i],linestyle=':')
    elif var == 'NN':
        for i, cn in enumerate(casenames.keys()):
            da = get_vars(ds[cn], 'tNN')
            da.sel(time=tslice).mean(dim='time').plot(ax=ax,y='z',color=colors[i],linestyle='--')
    elif var == 'ww':
        for i, cn in enumerate(casenames.keys()):
            da = get_vars(ds[cn], 'tke')
            da.sel(time=tslice).mean(dim='time').plot(ax=ax,y='z',color=colors[i],linestyle='--')
    elif var == 'uu':
        for i, cn in enumerate(casenames.keys()):
            da = get_vars(ds[cn], 'vv')
            da.sel(time=tslice).mean(dim='time').plot(ax=ax,y='z',color=colors[i],linestyle='--')
    elif var == 'wt':
        for i, cn in enumerate(casenames.keys()):
            da = get_vars(ds[cn], 'wtsb')
            da.sel(time=tslice).mean(dim='time').plot(ax=ax,y='zi',color=colors[i],linestyle='--')
    for i, cn in enumerate(casenames.keys()):
        da = get_vars(ds[cn], var)
        if 'z' in da.coords:
            yaxis = 'z'
        elif 'zi' in da.coords:
            yaxis = 'zi'
        da.sel(time=tslice).mean(dim='time').plot(ax=ax,y=yaxis,color=colors[i])
    ax.set_xlim(xlims[var])
    ax.set_ylim([-depth,0])
    ax.set_ylabel('')
    ax.set_xlabel(xlabels[var])
    ax.text(xabc[lr[k]], yabc[tb[k]], '({:s})'.format(abc[k]), transform=ax.transAxes, va=topbottom[tb[k]], ha=leftright[lr[k]])
    ax.grid()
    
for i in np.arange(2):
    axarr[i,0].set_ylabel('$z$ [m]')

    ax = axarr[0,0]
ax.plot(np.nan, np.nan, color='k', label='$\overline{u}^L/w_*$')
ax.plot(np.nan, np.nan, color='k', linestyle=':', label='$\overline{u}/w_*$')
ax.plot(np.nan, np.nan, color='k', linestyle='--', label='$u^S/w_*$')
ax.legend(loc='lower right', framealpha=1)

ax = axarr[0,1]
ax.plot(np.nan, np.nan, color='k', label='$N^2/N^2_0$')
ax.plot(np.nan, np.nan, color='k', linestyle='--', label='$N_*^2/N^2_0$')
ax.legend(loc='center right', framealpha=1)

ax = axarr[0,2]
ax.plot(np.nan, np.nan, color='k', label='$\overline{w^\prime b^\prime}/B_0$')
ax.plot(np.nan, np.nan, color='k', linestyle='--', label='$q^{sgs}_b/B_0$')
ax.legend(loc='lower right', framealpha=1)

ax = axarr[1,0]
ax.plot(np.nan, np.nan, color='k', label='$\overline{{w^\prime}^2}/w_*^2$')
ax.plot(np.nan, np.nan, color='k', linestyle='--', label='TKE$/w_*^2$')
ax.legend(loc='center left', bbox_to_anchor=(0, 0.65), framealpha=1)

ax = axarr[1,1]
ax.plot(np.nan, np.nan, color='k', label='$\overline{{u^\prime}^2}/w_*^2$')
ax.plot(np.nan, np.nan, color='k', linestyle='--', label='$\overline{{v^\prime}^2}/w_*^2$')
ax.legend(loc='center right', framealpha=1)

ax = axarr[1,-1]
for i, cn in enumerate(casenames.keys()):
    ax.plot(np.nan, np.nan, color=colors[i], label=labels[i])
ax.legend(ncol=3, loc='lower right', bbox_to_anchor=(0.3, -0.45))

# plt.tight_layout()
plt.subplots_adjust(top=0.97, bottom=0.18, left=0.09, right=0.97, hspace=0.3, wspace=0.15)
figname = os.path.join(figpath, 'profiles-ncar-cmp-wave-stratification_v2')
fig.savefig(figname, dpi = 300, facecolor='w')
